#Imports

In [0]:
# imports related to PyTorch
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict

# python tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# other
import json
from PIL import Image

# google collab specific
from google.colab import drive

#Loading of data
We are using `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The dataset is split into three parts, training, validation, and testing. For the training, there has been applyed transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this we don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks we'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three set we'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225], calculated from the ImageNet images. These values will shift each color channel to be centered at 0 and range from -1 to 1.

#### Gets acces to the google drive folder
\* only if running in google collab

In [0]:
# gets acces to the google drive folder
drive.mount('/content/gdrive')

#### getting the data
we are using the data loader class which means that the images are arranged in this way:

* class 'dog'
    * root/dog/xxx.png
    * root/dog/xxy.png
    * root/dog/xxz.png
* class 'cat'
    * root/cat/123.png
    * root/cat/nsdf3.png
    * root/cat/asd932_.png



In [0]:
# paths to data needs to get changed, for new data location 
dir = '/content/gdrive/My Drive/Colab Notebooks/PyTorch-project/flowers'

data_dir = {
    'train': '/train',
    'valid':'/valid',
    'test': '/test'   
}

# Defines the transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
                transforms.RandomRotation(30),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]),
    'valid': transforms.Compose([
                transforms.Resize(255),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]),
    'test': transforms.Compose([
                transforms.Resize(255),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
}


# Loads the datasets with ImageFolder
data_datasets = {
    'train': datasets.ImageFolder(data_dir['train'], transform=data_transforms['train']),
    'valid': datasets.ImageFolder(data_dir['valid'], transform=data_transforms['valid']),
    'test': datasets.ImageFolder(data_dir['test'], transform=data_transforms['test'])
}
    
# Using the image datasets and the trainforms, defines the dataloaders
data_dataloaders = {
    'train': torch.utils.data.DataLoader(data_datasets['train'], batch_size=32, shuffle=True),
    'valid': torch.utils.data.DataLoader(data_datasets['valid'], batch_size=32),
    'test': torch.utils.data.DataLoader(data_datasets['test'], batch_size=32)
}